In [1]:
import fundamentus as fd
import pandas as pd
import numpy as np
#import warnings
#from io import StringIO
import streamlit as st

2024-03-25 15:49:36,620 [logging.log_init] INFO: LOGLEVEL=INFO


In [10]:
vol_min = 400000
valor_total = 10000
# Removendo financeiras menos WIZC3
df= fd.get_resultado_raw();

for i in range(1, 51):
    fin = fd.list_papel_setor(i);  # finance
    if 'BBAS3' in fin:
        fin = fin;
        break

for i in range(1, 51):
    seg = fd.list_papel_setor(i);  # finance
    if 'WIZC3' in seg:
        financeiras = fin + seg;
        break

financeiras.remove('WIZC3')

df = df[~df.index.isin(financeiras)]

# Removendo ADRs e BDRs

df = df[~(df.index.astype(str).str.contains('33') | df.index.astype(str).str.contains('34'))]
df.head()

C:\Users\Gustavo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\fundamentus\resultado.py:50: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(content.text, decimal=",", thousands='.')[0]
C:\Users\Gustavo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\fundamentus\setor.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(content.text, decimal=",", thousands='.')[0]
C:\Users\Gustavo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\fundamentus\setor.py:42: FutureWarning: Passing literal html 

Multiples,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
papel,,,,,,,,,,,,,,,,,,,,
AALR3,8.56,-4.44,1.10,0.858,0.0000,0.379,-4.82,25.49,-0.84,46.30,12.02,0.0337,-0.1853,0.71,0.0171,-0.2484,747766.0,9.174560e+08,1.14,0.0351
ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1450,0.0,5.871730e+09,0.00,0.4586
ABEV3,12.41,13.48,2.48,2.452,0.0588,1.474,-44.01,11.49,-12.27,10.74,7.93,0.2133,0.1876,0.89,0.1790,0.1836,334666000.0,7.896930e+10,0.04,0.1211
ABYA3,4.91,-214.80,1.76,2.055,0.0000,0.527,1.98,19.96,-2.75,33.67,33.67,0.1029,-0.0096,2.09,0.0278,-0.0082,0.0,2.920600e+08,1.31,0.1641
ACES3,95.27,8.79,2.28,1.693,0.0000,1.438,6.40,7.20,20.77,6.54,6.54,0.2352,0.1927,2.05,0.2690,0.2595,0.0,3.105800e+09,0.09,0.1334


In [6]:


# Mantendo Margem Ebit Apenas Maior que Zero

df = df.loc[df['Mrg Ebit'] > 0]

# Mantendo Volume Financeiro Apenas Maior que 500k a cada 2 meses

df = df.loc[df['Liq.2meses'] >= vol_min]
#st.write(df['Liq.2meses'].unique())

# Step 1: Extract the first 4 characters from the index
df['First4Chars'] = df.index.str[:4]

# Step 2: Determine duplicates in the first 4 characters of the index
duplicates = df.duplicated(subset='First4Chars', keep=False)

# Step 3: For each group of duplicates, identify the row with the maximum 'Liq.2meses' value
max_values = df.groupby('First4Chars')['Liq.2meses'].transform('max')

# Step 4: Filter the DataFrame to keep only the rows with the maximum 'Liq.2meses' value for each group of duplicates
filtered_df = df[(~duplicates) | (df['Liq.2meses'] == max_values)]

# Drop the temporary column used for grouping
filtered_df = filtered_df.drop(columns='First4Chars')

sorted_df = filtered_df.sort_values(by=['EV/EBIT', 'Liq.2meses'], ascending=[True, False])

sorted_df['Quantidade'] = round((valor_total/20)/sorted_df['Cotação'],0)

sorted_df['Valor'] = sorted_df['Quantidade'] * sorted_df['Cotação']

sorted_df = sorted_df[['Cotação','EV/EBIT', 'Liq.2meses', 'Quantidade', 'Valor']].head(20)